In [1]:
# Title create_historical_traffic.ipynb
# Description: This script will generate and copy the traffic data to the 
#              container and add the predicted traffic to the routing graph.
# Note: This notebook is intended to be run after container is built and running
# Last Modified: 2024-09-23

In [2]:
# Build and run the docker container if not already running
#! sudo docker build -t valhalla . && docker run -dt --name valhalla -p 8002:8002 valhalla

In [3]:
import pandas as pd
import os
import subprocess
TRAFFIC_FOLDER_PATH = "./data/traffic" # must match valhalla_add_predicted_traffic.sh
SAMPLE_SPEED = "BbwACv/mAA//8gAK//kACwAGABcABQAaABwAWQD+/64AF//pABH/7gAI////+f/6//j//f/vAA4AU//c//T/4//2/+//+v/vAAP/7AAK/+YAB/+3/7MASwAPABcABwAOAAEAAgACAAH/8//+//X//QATAAD/+QAO//sABgAGABEABgAUAAUAEwALAAn/3QAJAAL/+AAB//f//f/3//T//v//AAD/9gAz/+r/0wAC//YAB/////8AAAAN//gAA//7AAP/7AAMAAcABgAE//sAAAAB//7//QAD//kABv/3//4APgAH//kABP/+//wAA//3AAH//AAE//YABAAA/+n/9v////8ABQAD//8ACgAAAAcABP/8AAL/6f/aAB4ABAAF//3////6//4AAP/3AAEAB//6ABUAHf/rAAAAA//8AAwAAAACAAAADf/8AAUAAAAC//f/+f/6AAAAAf/4AAH////+//8AA//+AAgAAQAB//kABf/4AAT//P/9//3/+v/6//7/+v/5//YAFAAM//8ABw=="

In [4]:
# run ./valhalla_ways_to_ids in the container to regenerate the ways_to_ids.csv file
#! docker exec -it valhalla /bin/bash -c /custom_files/valhalla_ways_to_ids /custom_files/conf/valhalla.json"

# copy the ways_to_ids.csv file from the container to the host
# os.system('docker cp valhalla:/custom_files/valhalla_tiles/way_edges_ids_paths.csv ./data/way_edges_ids_paths.csv')

In [5]:
# read the ways_to_ids.csv file

try: 
    df = pd.read_csv('./data/way_edges_ids_paths.csv')
except FileNotFoundError:
    print("Error: way_edges_ids_paths.csv file not found. Attempting to copy the file from the container.")
    os.system('docker cp valhalla:/custom_files/valhalla_tiles/way_edges_ids_paths.csv ./data/way_edges_ids_paths.csv')
    df = pd.read_csv('./data/way_edges_ids_paths.csv')
    
print(len(df))
df.head()

3555513


,osm_id,edge_id,graph_id,path
0,6845223,145531392122,2/727567/4337,2/000/727/567.csv
1,6845223,145632055418,2/727567/4340,2/000/727/567.csv
2,16120594,145397174394,2/727567/4333,2/000/727/567.csv
3,16120594,145430728826,2/727567/4334,2/000/727/567.csv
4,16120594,145464283258,2/727567/4335,2/000/727/567.csv


In [32]:
# =================== SPEED DATA =================== #
# set freeflow & constrained speeds 
#       thelse are the (default speeds for night and day
#       used as a fallback if there is no speed data for a time period
# set predicted speeds
#       the speeds are predicted for each 5 minute interval for a week
#       2016 speeds, with 288 values for each day of the week
#       speeds are in km/h and must be higher than 5 to be taken into account
#       speeds must be encoded using the valhalla_encode_speed executable

df['freeflow_speed'] = 10
df['constrained_speed'] = 10

#list of 2016 speeds SUNDAY (1st 288 values are 10, the rest are 100)
test_speeds = [10]*288 + [100]*1728
#run the valhalla_encode_speed executable 
encoded_speed = subprocess.check_output(["docker", "exec", "valhalla", "/custom_files/valhalla_encode_speed", str(test_speeds)[1:-1]], text=True)

df['encoded_speeds'] = encoded_speed


In [33]:
# groupby path and write to csv
for name, group in df.groupby('path'):
    os.makedirs(f"{TRAFFIC_FOLDER_PATH}/{name[:-7]}", exist_ok=True)
    group[['graph_id', 'freeflow_speed', 'constrained_speed', 'encoded_speeds']].to_csv(f"{TRAFFIC_FOLDER_PATH}/{name}", index=False, header=False)

In [ ]:
# add the traffic data to the container
os.system('./valhalla_add_predicted_traffic.sh')

In [35]:
# TESTING THE TRAFFIC DATA
import requests
import folium
import random
import time

In [36]:
time.sleep(5) # wait for the container if Run All

In [88]:
# https://valhalla.github.io/valhalla/api/turn-by-turn/api-reference/
# date_time
#  type 0 - Current departure time
#  type 1 - Specified departure time
#  type 2 - Specified arrival time
#  type 3 - Invariant specified time. Time does not vary over the course of the path.
#  value - ISO 8601 format (YYYY-MM-DDThh:mm) in the local time zone of departure or arrival. For example "2016-07-03T08:06"

# speed_types: freeflow, constrained, predicted, or current

def send_req(time):
    return requests.post('http://localhost:8002/sources_to_targets', json={
        #"date_time": {"type": "2", "value": time},
        "sources": [
            {"lat": 36.131822, "lon": -86.668783, "date_time": time},
            {"lat": 36.144985, "lon": -86.793394, "date_time": time},
            {"lat": 36.109791, "lon": -86.725040, "date_time": time},
        ],
        "targets": [
            {"lat": 36.131822, "lon": -86.668783, "date_time": time},
            {"lat": 36.144985, "lon": -86.793394, "date_time": time},
            {"lat": 36.109791, "lon": -86.725040, "date_time": time},
        ],
        "costing":"auto",
        "recostings":[
            {"costing":"auto","fixed_speed":20,"name":"auto_20"},
        ]
        #"speed_types":"predicted",
    })

times = ["2024-09-21T07:00", "2024-09-22T08:00", "2024-09-23T09:00"]
trips = []
for time_ in times:
    res = send_req(time_).json()
    trips.append(res)
    print(res.get('warnings',[{'text':0}])[0].get('text',0))
    print(res['sources_to_targets'])

0
[[{'distance': 0.0, 'time': 0, 'to_index': 0, 'from_index': 0}, {'distance': 14.398, 'time': 10238, 'to_index': 1, 'from_index': 0}, {'distance': 9.616, 'time': 6797, 'to_index': 2, 'from_index': 0}], [{'distance': 14.396, 'time': 10238, 'to_index': 0, 'from_index': 1}, {'distance': 0.0, 'time': 0, 'to_index': 1, 'from_index': 1}, {'distance': 8.673, 'time': 6177, 'to_index': 2, 'from_index': 1}], [{'distance': 9.616, 'time': 6797, 'to_index': 0, 'from_index': 2}, {'distance': 8.673, 'time': 6177, 'to_index': 1, 'from_index': 2}, {'distance': 0.0, 'time': 0, 'to_index': 2, 'from_index': 2}]]
0
[[{'distance': 0.0, 'time': 0, 'to_index': 0, 'from_index': 0}, {'distance': 14.398, 'time': 10238, 'to_index': 1, 'from_index': 0}, {'distance': 9.616, 'time': 6797, 'to_index': 2, 'from_index': 0}], [{'distance': 14.396, 'time': 10238, 'to_index': 0, 'from_index': 1}, {'distance': 0.0, 'time': 0, 'to_index': 1, 'from_index': 1}, {'distance': 8.673, 'time': 6177, 'to_index': 2, 'from_index': 1

In [30]:
inv = 1.0 / 1e6;
def decode(encoded):
  decoded = []
  previous = [0,0]
  i = 0
  while i < len(encoded):
    ll = [0,0]
    for j in [0, 1]:
      shift = 0
      byte = 0x20
      while byte >= 0x20:
        byte = ord(encoded[i]) - 63
        i += 1
        ll[j] |= (byte & 0x1f) << shift
        shift += 5
      ll[j] = previous[j] + (~(ll[j] >> 1) if ll[j] & 1 else (ll[j] >> 1))
      previous[j] = ll[j]
    decoded.append([float('%.6f' % (ll[0] * inv)), float('%.6f' % (ll[1] * inv))])
  return decoded

encoded_shapes = [trip['trip']['legs'][0]['shape'] for trip in trips]
encoded_shapes = list(set(encoded_shapes))

shapes = [decode(encoded_shape) for encoded_shape in encoded_shapes]

#make a small map
f = folium.Figure(width=1000, height=500)
m = folium.Map(location=[36.15, -86.74], zoom_start=12, min_zoom = 12).add_to(f)
for shape in shapes:
    #set a color for each trip
    color = "#{:06x}".format(random.randint(0, 0xFFFFFF))
    folium.PolyLine(
        locations=shape,
        color=color,
        weight=5,
        tooltip="",
        opacity=0.8
    ).add_to(m)
m